# Download Libs

In [88]:
#import nltk
#nltk.download('all')

In [89]:
#pip install -U spacy

# Restart Runtime

In [90]:
#!python3 -m spacy download en_core_web_trf

In [91]:
# pip install spacy-transformers -f https://download.pytorch.org/whl/torch_stable.html

# Download Data

In [92]:
#from google.colab import files
#files.upload() #enviar o arquivo kaggle.json

#antes de importar o dataset nós iremos provisionar o local de armazenamento
#Alterar a permissão para evitar o aviso durante a partida da ferramenta Kaggle
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

!rm ./data/*
!kaggle competitions download -c sentiment-analysis-on-movie-reviews
!unzip ./sentiment-analysis-on-movie-reviews.zip -d ./data/
!rm sentiment-analysis-on-movie-reviews.zip
!unzip ./data/test.tsv.zip -d ./data/
!unzip ./data/train.tsv.zip -d ./data/
!rm ./data/train.tsv.zip
!rm ./data/test.tsv.zip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 53%|████████████████████   

# Read Data

In [93]:
import pandas as pd

test = pd.read_csv('./data/test.tsv', sep='\t')
train = pd.read_csv('./data/train.tsv', sep='\t')
# sample_submission = pd.read_csv('./data/sampleSubmission.csv', sep='\t')

In [94]:
train = train.drop_duplicates(subset = "SentenceId")

In [95]:
train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
63,64,2,"This quiet , introspective and entertaining in...",4
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
116,117,4,A positively thrilling combination of ethnogra...,3
156,157,5,Aggressive self-glorification and a manipulati...,1
...,...,...,...,...
155984,155985,8540,... either you 're willing to go with this cla...,2
155997,155998,8541,"Despite these annoyances , the capable Claybur...",2
156021,156022,8542,-LRB- Tries -RRB- to parody a genre that 's al...,1
156031,156032,8543,The movie 's downfall is to substitute plot fo...,1


# Tokenization

In [96]:
import nltk
import spacy
nlp = spacy.load("en_core_web_trf")

In [97]:
from nltk.corpus import stopwords
stopwords = [word for word in stopwords.words('english')] + [word for word in nlp.Defaults.stop_words]
stopwords = [word.lower() for word in stopwords]

def tokenize(text):
	doc = nlp(text)
	tokens = [token.orth_.lower() for token in doc if token.is_alpha and (token.orth_.lower() not in stopwords)]
	text_pre = None
	for token in tokens:
		if text_pre is None:
			text_pre = token
		else:
			text_pre = text_pre + ' ' + token
	return text_pre

In [98]:
def lemmatize(text):
	doc = nlp(text)
	tokens = [token.lemma_ for token in doc]
	text_lemma = None
	for token in tokens:
		if text_lemma is None:
			text_lemma = token
		else:
			text_lemma = text_lemma + ' ' + token
	return text_lemma

In [99]:
import nltk
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [100]:
def stematize(text):
	doc = nlp(text)	
	tokens = [stemmer.stem(token.orth_) for token in doc]
	text_stema = None
	for token in tokens:
		if text_stema is None:
			text_stema = token
		else:
			text_stema = text_stema + ' ' + token
	return text_stema

In [101]:
def pre_process(text):
	if text is None:
		return text
	else: 
		return stematize(lemmatize(text))

In [102]:
train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
63,64,2,"This quiet , introspective and entertaining in...",4
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
116,117,4,A positively thrilling combination of ethnogra...,3
156,157,5,Aggressive self-glorification and a manipulati...,1
...,...,...,...,...
155984,155985,8540,... either you 're willing to go with this cla...,2
155997,155998,8541,"Despite these annoyances , the capable Claybur...",2
156021,156022,8542,-LRB- Tries -RRB- to parody a genre that 's al...,1
156031,156032,8543,The movie 's downfall is to substitute plot fo...,1


In [103]:
train['Phrase'] = train['Phrase'].map(lambda text: pre_process(tokenize(text)))

/home/gfmco/.local/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
trainProcessed = train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,seri escapad demonstr adag good goos good gand...,1
63,64,2,quiet introspect entertain independ worth seek,4
81,82,3,fan ismail merchant work suspect hard time sit,1
116,117,4,posit thrill combin ethnographi intrigu betray...,3
156,157,5,aggress self glorif manipul whitewash,1
...,...,...,...,...
155984,155985,8540,will claustrophob concept,2
155997,155998,8541,despit annoy capabl clayburgh tambor great job...,2
156021,156022,8542,tri parodi genr joke unit state,1
156031,156032,8543,movi downfal substitut plot person,1


In [ ]:
trainProcessed.to_csv("./processedData/train.csv")
trainProcessed = pd.read_csv("./processedData/train.csv")

In [87]:
train

,Unnamed: 0,PhraseId,SentenceId,Phrase,Sentiment
0,0,1,1,A series of escapades demonstrating the adage ...,1
1,1,2,1,A series of escapades demonstrating the adage ...,2
2,2,3,1,A series,2
3,3,4,1,A,2
4,4,5,1,series,2
...,...,...,...,...,...
156055,156055,156056,8544,Hearst 's,2
156056,156056,156057,8544,forced avuncular chortles,1
156057,156057,156058,8544,avuncular chortles,3
156058,156058,156059,8544,avuncular,2
